Signal Processing (Part 4: Application)
---

Part นี้ จะลองเอาความรู้ทางด้าน signal processing มาเล่นกับไฟล์เสียงดู

In [1]:
import numpy as np
from numpy.random import rand
import librosa
import librosa.display
import soundfile as sf
from sklearn.decomposition import FastICA
import IPython.display as ipd

In [3]:
source1, sr1 = librosa.load('./data/source1.wav')
source2, sr2 = librosa.load('./data/source2.wav')

<ipython-input-3-9c376c50d454>:1: UserWarning: PySoundFile failed. Trying audioread instead.
  source1, sr1 = librosa.load('./data/source1.wav')


FileNotFoundError: ignored